In [ ]:
from __future__ import print_function
import numpy as np # For numerical fast numerical calculations
import matplotlib.pyplot as plt # For making plots
import pandas as pd # Deals with data
import seaborn as sns # Makes beautiful plots
import sys
import tensorflow as tf
from pandas import pandas as pd
import category_encoders as ce
import datetime
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import os
import glob
import numpy as np
import scipy as sp
import pandas as pd
# skimage
from skimage.io import imshow, imread, imsave
from skimage.transform import rotate, AffineTransform, warp,rescale, resize, downscale_local_mean
from skimage import color,data
from skimage.exposure import adjust_gamma
from skimage.util import random_noise
# imgaug
import imageio
import imgaug as ia
import imgaug.augmenters as iaa
# Albumentations
import albumentations as A 
# Keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator,array_to_img, img_to_array, load_img 
#visualisation
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#%matplotlib inline
import seaborn as sns
from IPython.display import HTML, Image
import cv2
import os
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import seaborn as sns

!pip install efficientnet
!pip install --upgrade wandb

from kaggle_datasets import KaggleDatasets
import efficientnet.tfkeras as efn

import wandb
from wandb.keras import WandbCallback



# load data

p_train=pd.read_csv('../input/plant-pathology-2020-fgvc7/train.csv')
p_test=pd.read_csv('../input/plant-pathology-2020-fgvc7/test.csv')

# Load datasets and choose image_size

In [ ]:
p_train=pd.read_csv('../input/plant-pathology-2020-fgvc7/train.csv')
p_test=pd.read_csv('../input/plant-pathology-2020-fgvc7/test.csv')

target = p_train[['healthy', 'multiple_diseases', 'rust', 'scab']]
test_ids = p_test['image_id']

img_size=224

In [ ]:
colors=['#78C850','#6890F0']
p_train['healthy'].value_counts().plot(kind='bar',title='Healthy Frequency Count',color=colors)

In [ ]:
p_train['multiple_diseases'].value_counts().plot(kind='bar',title='Multi-Diseases Frequency Count',color=colors)

In [ ]:
p_train['rust'].value_counts().plot(kind='bar',title='Rust Frequency Count',color=colors)

In [ ]:
p_train['scab'].value_counts().plot(kind='bar',title='Scab Frequency Count',color=colors)

In [ ]:
train_image=[]
for name in p_train['image_id']:
    path='../input/plant-pathology-2020-fgvc7/images/'+name+'.jpg'
    img=cv2.imread(path)
    image=cv2.resize(img,(img_size,img_size),interpolation=cv2.INTER_AREA)
    train_image.append(image)

fig, ax = plt.subplots(1, 4, figsize=(15, 15))
for i in range(4):
    ax[i].set_axis_off()
    ax[i].imshow(train_image[i])
    
    
test_image=[]
for name in p_test['image_id']:
    path='../input/plant-pathology-2020-fgvc7/images/'+name+'.jpg'
    img=cv2.imread(path)
    image=cv2.resize(img,(img_size,img_size),interpolation=cv2.INTER_AREA)
    test_image.append(image)
fig, ax = plt.subplots(1, 4, figsize=(15, 15))
for i in range(4):
    ax[i].set_axis_off()
    ax[i].imshow(test_image[i])    

from tensorflow.keras.preprocessing.image import img_to_array

x_train = np.ndarray(shape=(len(train_image), img_size, img_size, 3),dtype = np.float32)
i=0
for image in train_image:
    x_train[i]=img_to_array(image)
    x_train[i]=train_image[i]
    i=i+1
x_train=x_train/255
print('Train Shape: {}'.format(x_train.shape))

x_test = np.ndarray(shape=(len(test_image), img_size, img_size, 3),dtype = np.float32)
i=0
for image in test_image:
    x_test[i]=img_to_array(image)
    x_test[i]=test_image[i]
    i=i+1
    
x_test=x_test/255
print('Test Shape: {}'.format(x_test.shape))

In [ ]:
y = p_train.copy()
del y['image_id']
y.head()

y_train = np.array(y.values)
print(y_train.shape,y_train[0])

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=115)

x_train.shape, x_val.shape, y_train.shape, y_val.shape

In [ ]:
from imblearn.over_sampling import SMOTE 

sm = SMOTE(random_state = 115) 
 
x_train, y_train = sm.fit_resample(x_train.reshape((-1, img_size * img_size * 3)), y_train)
x_train = x_train.reshape((-1, img_size, img_size, 3))
x_train.shape, y_train.sum(axis=0)

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
LR_reduce=ReduceLROnPlateau(monitor='val_accuracy',
                            factor=.5,
                            patience=10,
                            min_lr=.000001,
                            verbose=1)

ES_monitor=EarlyStopping(monitor='val_loss',
                          patience=20)

reg = .0005

# Initialize TPU

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)

# Creating EfficientNetB7 Model

In [ ]:
def create_model():
    return tf.keras.models.Sequential([
        efn.EfficientNetB7(include_top=False, weights='imagenet', input_shape=(512, 512, 3)),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(4, activation='softmax')
    ])

In [ ]:
with strategy.scope(): # to use TPU while training
    model = create_model()
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC(name='AUC')])
    model.summary()

# Initialize wandb project

In [ ]:
## Initlisazie wandb project
wandb.init(project='Test 17052022',
           notes='Round 1',
           name='Training EfficientB7'
          )

In [ ]:
# Making the wandb callback
wandb_callback = WandbCallback()

# Data fitting

In [ ]:
datagen = ImageDataGenerator(rotation_range=45,
                             shear_range=.25,
                              zoom_range=.25,
                              width_shift_range=.25,
                              height_shift_range=.25,
                              rescale=1/255,
                              brightness_range=[.5,1.5],
                              horizontal_flip=True,
                              vertical_flip=True,
                              fill_mode='nearest'
#                              featurewise_center=True,
#                              samplewise_center=True,
#                              featurewise_std_normalization=True,
#                              samplewise_std_normalization=True,
#                              zca_whitening=True
                              )

# history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=24),
#                               epochs=300,
#                               steps_per_epoch=x_train.shape[0] // 24,
#                               verbose=1,
#                               callbacks=[ES_monitor,LR_reduce],
#                               validation_data=datagen.flow(x_val, y_val,batch_size=24),
#                               validation_steps=x_val.shape[0]//24
#                               )


history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                              steps_per_epoch=x_train.shape[0] // 32,
                              epochs=120,
                              verbose=1,
                              callbacks=[ES_monitor,LR_reduce],
                              validation_data=(x_val, y_val))

# Các bước xử lý kết quả khi chạy xong 1 mô hình

In [ ]:
test_part_train = model.predict(valid_dataset, verbose = 1)

In [ ]:
name_img = valid_img[0].split('/')[4]
img = plt.imread(f'../input/plant-pathology-2020-fgvc7/images/{name_img}')
# plt.imshow(img)

In [ ]:
test_around = np.zeros_like(test_part_train)
test_around[np.arange(len(test_part_train)), test_part_train.argmax(1)] = 1

In [ ]:
title_img_error = np.array([])
labels_error = np.array([[0, 0, 0, 0]])
labels_true = np.array([[0, 0, 0, 0]])

for i in range (0, valid_labels.shape[0]):
    check = test_around[i] == valid_labels[i]
    if False in check:
        title_img_error = np.append(title_img_error, valid_img[i].split('/')[4])
        labels_error = np.r_[labels_error, [test_around[i]]]
        labels_true = np.r_[labels_true, [valid_labels[i]]]

labels_error = np.delete(labels_error, (0), axis=0)
labels_true = np.delete(labels_true, (0), axis=0)

print(title_img_error.size)

In [ ]:
# NOTE: You can change the w, h, rows, cols to see another img.
w, h = 10, 10 # Setting width and height of every image
rows, cols = title_img_error.size//4, 4 # Setting the number of image rows & cols
# rows, cols = 5, 4

fig = plt.figure(figsize = (20, 14)) # Making the figure with size

plt.title('Images')
plt.axis('off')

#Going through every cell in rows adn cols
# 1 <= i <= cols*rows
for i in range(1, cols * rows + 1):
  # Reading the image
  img = plt.imread(f"../input/plant-pathology-2020-fgvc7/images/{title_img_error[i - 1]}")

  # Adding the subplot
  fig.add_subplot(rows, cols, i)
    
  title_img = ''

  # Setting the true labels of every image
  if labels_true[i - 1, 0] == 1:
    title_img += 'Healthy'
  elif labels_true[i -1, 1] == 1:
    title_img += 'Multiple Diseases'
  elif labels_true[i - 1, 2] == 1:
    title_img += 'Rust'
  else:
    title_img += 'Scab'
    
       # Setting the error labels of every image
  if labels_error[i - 1, 0] == 1:
    title_img += ' / Healthy'
  elif labels_error[i - 1, 1] == 1:
    title_img += ' / Multiple Diseases'
  elif labels_error[i - 1, 2] == 1:
    title_img += ' / Rust'
  elif labels_error[i - 1, 3] == 1:
    title_img += ' / Scab'
  else:
    title_img += ' / Unknow'
    
  plt.title(title_img)
  # Show the image
  plt.imshow(img)

  # Setting the axis off
  plt.axis("off")

#Showing the figure
plt.show()

# Predicting our Test Dataset - Tính kết quả với bộ test và xuất ra file

In [ ]:
test_data = pd.read_csv("../input/plant-pathology-2020-fgvc7/test.csv")

In [ ]:
test_imgs_path = GCS_PATH + '/images/' + test_data['image_id'] + '.jpg'

test_imgs_path[0]

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

test_dataset = (
                tf.data.Dataset
                .from_tensor_slices(test_imgs_path)
                .map(read_img, num_parallel_calls=AUTO)
                .batch(BATCH_SIZE))

In [ ]:
predictions = model.predict(test_dataset, verbose=1)

In [ ]:
predictions_around = np.zeros_like(predictions)
predictions_around[np.arange(len(predictions)), predictions.argmax(1)] = 1

In [ ]:
test_data['healthy'] = predictions_around[:, 0]
test_data['multiple_diseases'] = predictions_around[:, 1]
test_data['rust'] = predictions_around[:, 2]
test_data['scab'] = predictions_around[:, 3]
# test_data

In [ ]:
test_data

In [ ]:
test_data.to_csv('submission_test_17052022.csv', index=False)